# Adding Cloud-Optimized GeoTIFFs to the MAAP Dashboard

The following notebook steps through how to add a dataset to the MAAP Dashboard.

Note, there are 2 scenarios:

1. Adding a single Cloud-Optimized GeoTIFF (COG) and,
2. Adding many distinct COGs as a "mosaic" with mosaicJSON.

High-level, the steps are:

1. Inspect your COG(s) to understand the best rescale and colormap name parameters. Optionally create a mosaic.
2. Define a colormap. Colormaps provide mappings of data values to RGB values.
3. Create a PR to the datasets repo to add or update your dataset.

The MAAP dashboard has 3 environments:

1. Developer-in-test (DIT): https://biomass.dit.maap-project.org
2. Staging: https://uat.earthdata.nasa.gov/maap-biomass
3. Production: https://earthdata.nasa.gov/maap-biomass

These instructions will guide you towards adding your dataset to `biomass.dit.maap-project.org`. The MAAP Dashboard team will "promote" changes to staging and production periodically (release schedule forthcoming).

# Setup

In [ ]:
%%capture
!pip install rasterio rio-cogeo supermercado

In [ ]:
# import the MAAP package
import ipycmc
from maap.maap import MAAP

import glob
import json
import os
import matplotlib
import urllib

# create MAAP class
maap = MAAP(maap_host='api.ops.maap-project.org')

In [ ]:
project_dir = "/projects/shared-buckets/<your_name>/<project_dir>"

In [ ]:
# Search for files to include, use recursive if nested folders (common in DPS output)
files = glob.glob(os.path.join(project_dir, "*.tif"), recursive=False)
files = [os.path.basename(f) for f in files]
files

In [ ]:
my_tif = files[1]

In [ ]:
%%bash -s "$project_dir" "$my_tif"
rio cogeo validate $1/$2

In [ ]:
%%bash -s "$project_dir" "$my_tif"
gdalinfo $1/$2 -stats

## Create some parameters for the dynamic tiler.

These parameters will be pased to `titiler_url` for visualization.

Note the values below: `band_min`, `band_max` and `colormap_name` are set as the current defaults for biomass in Mg/hectare for the dashboard. For other datasets, users should inspect the output of the `gdalinfo` for `band_min` and `band_max` and modify the `colormap_name` as makes sense for the current dataset. This notebook includes section on what colormaps are available and how to configure different types of colormaps and legends.

In [ ]:
titiler_url = f"https://titiler.maap-project.org"
band_min=0
band_max=400
rescale = f"{band_min},{band_max}"
bidx = 1
colormap_name = 'gist_earth_r'

## Helper functions

In [ ]:
def generate_s3_upload_path(uuid, file):
    filename = os.path.basename(file)
    bucket = maap.config['aws']['user_upload_bucket']
    directory = maap.config['aws']['user_upload_directory']
    location = f"s3://{bucket}/{directory}/{uuid}/{filename}"
    return location

def upload_and_return_location(files):
    response = maap.uploadFiles(files)
    uuid = response\
      .replace("Upload file subdirectory: ", "")\
      .replace(" (keep a record of this if you want to share these files with other users)", "")
    return [generate_s3_upload_path(uuid, file) for file in files]

# Step 1, Scenario 1: Adding a single COG

## 1. Upload the file

Only use the following steps if you only have one COG to share to the dashboard. If you want to create a mosaic from multiple COGs, skip to Step 1 Option 2.

Upload the file to S3 and make note of the location.

In [ ]:
location = upload_and_return_location([f"{project_dir}/{my_tif}"])[0]
s3_path = location.replace('s3://', '')
location

## 2. Test the COG with the titiler and ipyCMC

In [ ]:
## Build a WMTS call
wmts_url = f"x{titiler_url}/cog/WMTSCapabilities.xml"
params = {
    "tile_format": "png",
    "tile_scale": "1",
    "pixel_selection": "first",
    "TileMatrixSetId": "WebMercatorQuad",
    "url": location,
    "bidx": bidx, # Select which band to use
    "resampling_method":"nearest",
    "rescale": rescale,
    "return_mask": "true",
    "colormap_name": colormap_name # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

In [ ]:
w = ipycmc.MapCMC()
w

In [ ]:
wmts_link = wmts_call.replace('xhttps', 'https')
w.load_layer_config(wmts_link, "wmts/xml")

# Step 1, Scenario 2: Adding data from multiple COGs by creating a mosaic

Many datasets are comprised of many tiles distributed spatially over the globe. In order to visualize them all together, we can use [mosaicJSON](https://github.com/developmentseed/mosaicjson-spec) to create a mosaic for the dynamic tiler API. The dynamic tiler API knows how to read this mosaicJSON and select which tiles to render based on the current zoom, x and y coordinates across spatially distinct COGs.

In [ ]:
%%capture
!pip install cogeo-mosaic

In [ ]:
import glob
import os
import urllib

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [ ]:
# Skipping this step since we don't want to upload these large files more than once!
full_path_files = [f'{project_dir}/{file}' for file in files]
# maap.uploadFiles(full_path_files)

In [ ]:
uuid = 'xxx'
tiles = [generate_s3_upload_path(uuid, file) for file in files]
print(tiles)

In [ ]:
mosaicdata = MosaicJSON.from_urls(tiles, minzoom=1, maxzoom=16)

## Using mosaicJSON with titiler

There are 2 options for using mosaicJSON with titiler:

1. (Preferred) Post mosaicJSON to titiler `mosaics` endpoint and use the `mosaicjson/mosaics` endpoint for dynamic tiling.
2. Upload mosaicJSON to S3 and pass the S3 url to the titiler `mosaicjson/tiles` endpoint.

### Option 1: Post mosaicJSON to titiler

In [ ]:
import requests
from pprint import pprint

r = requests.post(
    url=f"{titiler_url}/mosaicjson/mosaics",
    headers={
        "Content-Type": "application/vnd.titiler.mosaicjson+json",
    },
    json=mosaicdata.dict(exclude_none=True)).json()

pprint(r)

In [ ]:
mosaic_link = list(filter(lambda x: x['rel'] == 'tiles', r['links']))[0]['href']
wmts_mosaic_link = list(filter(lambda x: x['rel'] == 'wmts', r['links']))[0]['href']

## Test your mosaic

In [ ]:
params = {
    "tile_format": "png",
    "bidx": bidx, # Select which band to use
    "resampling_method":"nearest",
    "rescale": rescale, 
    "return_mask": "true",
    "colormap_name": colormap_name # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_mosaic_link, urllib.parse.urlencode(params)])

w.load_layer_config(wmts_call, "wmts/xml")

### Option 2: Upload mosaicJSON to S3 and pass the S3 url to titiler `tiles` endpoint

In [ ]:
tilefile_name = "mosaic.json"
with MosaicBackend(tilefile_name, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

In [ ]:
mosaic_file_location = upload_and_return_location([tilefile_name])
mosaic_file_location

## Test your mosaic

In [ ]:
wmts_url = f"x{titiler_url}/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format": "png",
    "tile_scale": "1",
    "pixel_selection": "first",
    "TileMatrixSetId": "WebMercatorQuad",
    "url": mosaic_file_location[0],
    "bidx": bidx, # Select which band to use
    "resampling_method":"nearest",
    "rescale": rescale, 
    "return_mask": "true",
    "colormap_name": colormap_name # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

In [ ]:
wmts_link = wmts_call.replace('xhttps', 'https')
w.load_layer_config(wmts_link, "wmts/xml")

# Step 2. Define a color map

By default, the image will be displayed in greyscale if no `colormap_name` parameter is passed to the titiler API. Guidance below is provided to help determine what a valid colormap_name might be and how to create a legend for the dashboard.

## Dashboard ColorRamps & Legends

When using the dashboard, there 2 components for implementing a color scheme for your map. There is the map render and there is the legend.

> Titiler used for Cloud Optimized Geotiff (COG) rendering accepts any color scheme from the python matplotlib library, and custom color formulas.

* [Rio Tiler Colors](https://cogeotiff.github.io/rio-tiler/colormap/)
* [Matplotlib Colors](https://matplotlib.org/stable/tutorials/colors/colormaps.html) 

Available `colormap_name` values for titiler: `above, accent, accent_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, blues, blues_r, bone, bone_r, brbg, brbg_r, brg, brg_r, bugn, bugn_r, bupu, bupu_r, bwr, bwr_r, cfastie, cividis, cividis_r, cmrmap, cmrmap_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, dark2, dark2_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnbu, gnbu_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, greens, greens_r, greys, greys_r, hot, hot_r, hsv, hsv_r, inferno, inferno_r, jet, jet_r, magma, magma_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, oranges, oranges_r, orrd, orrd_r, paired, paired_r, pastel1, pastel1_r, pastel2, pastel2_r, pink, pink_r, piyg, piyg_r, plasma, plasma_r, prgn, prgn_r, prism, prism_r, pubu, pubu_r, pubugn, pubugn_r, puor, puor_r, purd, purd_r, purples, purples_r, rainbow, rainbow_r, rdbu, rdbu_r, rdgy, rdgy_r, rdpu, rdpu_r, rdylbu, rdylbu_r, rdylgn, rdylgn_r, reds, reds_r, rplumbo, schwarzwald, seismic, seismic_r, set1, set1_r, set2, set2_r, set3, set3_r, spectral, spectral_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r, tab20c, tab20c_r, terrain, terrain_r, twilight, twilight_r, twilight_shifted, twilight_shifted_r, viridis, viridis_r, winter, winter_r, wistia, wistia_r, ylgn, ylgn_r, ylgnbu, ylgnbu_r, ylorbr, ylorbr_r, ylorrd, ylorrd_r`


## Example 1: Class based known colors

In this example, the raster represents classes of forest with 11 possible values. There are specific colors selected to correspond to each class. We combine the list of colors and the list of classes and format them for the legend parameter the dashboard needs.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/taiga-forest-classification.json

In [ ]:
colors = [
    '#5255A3','#1796A3','#FDBF6F','#FF7F00', '#FFFFBF','#D9EF8B','#91CF60','#1A9850', '#C4C4C4','#FF0000', '#0000FF'
]
labels = [
    'Sparse & Uniform',
    'Sparse & Diffuse-gradual',
    'Sparse & Diffuse-rapid',
    'Sparse & Abrupt ',
    'Open & Uniform ',
    'Open & Diffuse-gradual',
    'Open & Diffuse-rapid',
    'Open & Abrupt',
    'Intermediate & Closed',
    'Non-forest edge (dry)',
    'Non-forest edge (wet)'
]

legend = [dict(color=colors[i],label=labels[i]) for i in range(0, len(colors))]
print(json.dumps(legend, indent=2))

# Copy and Paste the output below to your dashboard config.

## Example 2: Discrete ColorRamp

In this example, the range of values is known, but the color scale has many non-sequential colors. Starting with the premade color list, we create a continuous color ramp that uses the known colors as stops points. Arbitrarly 12 breaks looked decent in the dashboard legend so we split it into 12 discrete colors. Then combine the list of values and colors into the correct json syntax.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/ATL08.json

In [ ]:
forest_ht = matplotlib.colors.LinearSegmentedColormap.from_list('forest_ht', ['#636363','#FC8D59','#FEE08B','#FFFFBF','#D9EF8B','#91CF60','#1A9850','#005A32'], 12)
cols = [matplotlib.colors.to_hex(forest_ht(i)) for i in range(forest_ht.N)]

cats = range(0,25, (25//len(cols)))
legend = [[cats[i],cols[i]] for i in range(0, len(cols))]
text = (json.dumps(legend, separators=(',', ': ') ))

print(text.replace('],[','],\n['))
 
# Copy and Paste the output below to your dashboard config.

## Example 3: Continuous ColorRamp

In this example, we are using a built in ColorRamp from matplotlib. So we just need to extract enough colors to fill the legend adequately, and convert the colors to hex codes.

https://github.com/MAAP-Project/dashboard-datasets-maap/blob/main/datasets/topo.json

In [ ]:
cmap_name = 'gist_earth_r'
cmap = matplotlib.cm.get_cmap(cmap_name, 12)
cols = [matplotlib.colors.to_hex(cmap(i)) for i in range(cmap.N)]
print(cols)

# Copy and Paste the output below to your dashboard config.

# Step 3: Create and submit your dashboard dataset json

In [ ]:
# This example is for a continuous color ramps
dataset_id = "paraguay-estimated-biomass"
dataset_name = "Estimated Biomass in Paraguay"
dataset_type = "raster"
legend_type = "gradient-adjustable"
info = "Estimated biomass within 6km grids."
stops = cols
parameters = f"colormap_name={cmap_name}&rescale={band_min},{band_max}&bidx={bidx}"

In [ ]:
# Single COG
tiles_link = f"{{titiler_server_url}}/cog/tiles/{{z}}/{{x}}/{{y}}.png?url={location}&{parameters}"

In [ ]:
# Mosaic via mosaic json POST 
# https://titiler.maap-project.org/mosaicjson/mosaics/897be444-51c8-4c75-b493-ae5aacd431c5/tiles/{z}/{x}/{y}?bidx=1&max_size=1024&resampling_method=nearest&rescale=0,400&return_mask=true&colormap_name=gist_earth_r
tiles_link = f"{mosaic_link}?{parameters}"

In [ ]:
# Mosaic via mosaic json upload
tiles_link = f"{{titiler_server_url}}/mosaicjson/tiles/WebMercatorQuad/{{z}}/{{x}}/{{y}}.png?url={location}&{parameters}"

In [ ]:
dataset_dict = {
    "id": dataset_id,
    "name": dataset_name,
    "type": dataset_type,
    "swatch": {
      "color": "#6976d7",
      "name": "Moody Blue"
    },
    "source": {
        "type": dataset_type,
        "tiles": [ tiles_link ]
    },
    "legend": {
      "type": legend_type,
       "min": band_min,
       "max": band_max,
      "stops": stops
    },
    "info": info
}
print(json.dumps(dataset_dict, indent=4))

## Create a PR to the datasets repo

```bash
git clone git@github.com:MAAP-Project/biomass-dashboard-datasets.git
cd biomass-dashboard-datasets
git checkout -b ab/add-dataset
# select and copy json above
echo <copied_json> >> datasets/paraguay-estimated-biomass.json
```

### Add to the datasets list in config.yml

In `config.yml`:

```yaml
DATASETS:
- paraguay-estimated-biomass.json
```

### Add to the product or country pilot

In `country_pilots/paraguay/country_pilot.json`:
```json
{
    "id": "paraguay",
    "label": "Paraguay",
    //...
    "datasets": [
        {
            "id": "paraguay-forest-mask"
        },
        {
            "id": "paraguay-tree-cover"
        },
        {
            "id": "paraguay-estimated-biomass"
        }
    ]
}
```

## Add content to the summary

There should be a `summary.html` file corresponding to the product or country pilot you are working on, for example: `country_pilots/paraguay/summary.html`. Add or modify content in that file as appropriate.

## Submit a PR

Once you have added the dataset json file and summary content, submit a PR to https://github.com/MAAP-Project/biomass-dashboard-datasets. A member of the data team will review the PR and when it is merged your content will appear in biomass.dit.maap-project.org.